In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint
import time

from api_key import g_key
gmaps.configure(api_key = g_key)

In [2]:
Weather_Data = "../WeatherPy/Weather_CSV/weather_data.csv"

weather_df = pd.read_csv(Weather_Data)

weather_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Richards Bay,86,ZA,1603234156,71,-28.78,32.04,67.53,12.39
1,1,Mataura,30,NZ,1603233939,66,-46.19,168.86,55.00,8.99
2,2,Yellowknife,40,CA,1603233888,85,62.46,-114.35,28.99,12.75
3,3,Mar del Plata,14,AR,1603234099,58,-38.00,-57.56,53.60,8.05
4,4,Vaini,75,TO,1603234156,64,-21.20,-175.20,75.20,11.41


In [3]:
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df['Humidity'].astype(int)

map = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=15,
                                 point_radius=1)

map.add_layer(heat_layer)

map

Figure(layout=FigureLayout(height='420px'))

In [4]:
city_df = pd.DataFrame(weather_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (city_df["Max Temp"] <= 80) & (city_df["Max Temp"] > 70)
wind_speed = city_df["Wind Speed"] < 10
cloudiness = city_df["Cloudiness"] == 0

city_df = city_df.dropna()
len(city_df)

city_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
30,Muscat,71.60,4.70,0
36,Eyl,79.59,6.55,0
124,Kiryat Gat,75.00,1.01,0
130,Fayaoué,76.15,6.82,0
156,Hīt,74.07,9.24,0
223,Senanga,71.83,5.41,0
234,Bandarbeyla,78.35,8.08,0
263,Misratah,70.86,7.43,0
288,Kutum,74.10,8.50,0
294,Tikrit,72.14,9.13,0


In [5]:
hotel_name_df = pd.DataFrame(weather_df, columns=["City", "Country", "Max Temp", "Lat", "Lng"])
hotel_name_df["Hotel Name"] = ""
hotel_name_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Richards Bay,ZA,67.53,-28.78,32.04,
1,Mataura,NZ,55.00,-46.19,168.86,
2,Yellowknife,CA,28.99,62.46,-114.35,
3,Mar del Plata,AR,53.60,-38.00,-57.56,
4,Vaini,TO,75.20,-21.20,-175.20,
...,...,...,...,...,...,...
551,Nuuk,GL,35.60,64.18,-51.72,
552,Morondava,MG,76.68,-20.28,44.28,
553,Manaus,BR,86.00,-3.10,-60.02,
554,Coquimbo,CL,57.20,-29.95,-71.34,


In [ ]:
hotel = []
n = 0
ltt = hotel_name_df['Lat']
ling = hotel_name_df['Lng']


for index, city in hotel_name_df["City"]:
    target_coordinates =  f"{ltt},{ling}"
    
    params = {
    "location": target_coordinates,
    "keyword": "hotel",
    "radius": 50000,
    "type": "hotel",
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    print(f"Getting Results for City {city}.")
    
    response = requests.get(base_url, params=params)
    response = response.json()
    
    try:
        
        print(f" Hotel in {city}: {response[0]['name']}.")
        hotel_name_df.loc[index, "Hotel Name"] = response['results'][0]['name']

    except(KeyError, IndexError):
        print("Missing field/result... skipping.")
        
        print("------------")
    
    time.sleep(1)

print("-------End of Search-------")

In [ ]:
hotels = hotel_name_df["Hotel Name"].tolist()

In [ ]:
hotel_locations = hotel_name_df[["Lat", "Lng"]]
fig = gmaps.figure()
markers = gmaps.marker_layer(hotel_locations,
            info_box_content =[f"Hotel Name: {name}" for name in hotels ])
fig.add_layer(markers)
fig